In [ ]:
# using GuSTO"
include("../src/GuSTO.jl")

In [ ]:
robot = Astrobee3D()
model = AstrobeeSE3Manifold()
env = ISSCorner();

N = 200
tf_guess = 10.
goal_set = GoalSet()

# add_obstacles!(env)

# x_min_obs = [9.7, -0.1, 4.7]
# x_max_obs = [9.8, 0.1, 4.8]
# push!(env.obstacle_set, HyperRectangle(Vec3f0(x_max_obs), Vec3f0(x_max_obs-x_min_obs)))

# Small corner maneuver
# r_init = [9.2; 0.8; 4.2]
# r_goal = [10.9; 2.0; 5.0]

# Small free maneuver
# r_init = [9.2; 0.8; 4.2]
# r_goal = r_init + 1.*[1.; -0.25; 0.5]

# Straight free maneuver
# r_init = [9.2; 0.; 4.8]
# r_goal = r_init + [1.5; 0.; 0.]

# r_init = [10.2; -0.8; 5.6]
# r_goal = [10.2; 6.9; 4.2]

r_init = [1.; 0.; 0.]
r_goal = r_init + [1.; 0.; 0.]

# x_init = [r_init; zeros(3); quat2mrp(sqrt(1/3)*[1.; 0.; 1.; 1.]); zeros(3)]
# x_goal = [r_goal; zeros(3); quat2mrp([-0.5; 0.5; -0.5; 0.5]); zeros(3)]
# x_init = [r_init; zeros(3); GuSTO.quat2mrp([0.; 0.; 0.; 1.]); zeros(3)]
# x_goal = [r_goal; zeros(3); GuSTO.quat2mrp([0.; 0.; 0.; -1.]); zeros(3)]

q_init = [1.; 0.; 0.; 0.]
# q_goal = [0.5;0.5;0.5;1.]
# q_goal = [0.; 0.; 0.01; 1.]
q_goal = [1.; 0.; 0.; 0.]
# q_goal = [1.; 0.2; 0.3; 0.4]
q_goal = q_goal/norm(q_goal)

x_init = [r_init; zeros(3); q_init; zeros(3)]
# x_goal = [r_goal; zeros(3); [0.; 0.; 0.; 1.]; zeros(3)]
x_goal = [r_goal; zeros(3); q_goal; zeros(3)]
add_goal!(goal_set, Goal(PointGoal(x_goal), tf_guess, model))

PD = ProblemDefinition(robot, model, env, x_init, goal_set);
TOP = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOS_SCP = TrajectoryOptimizationSolution(TOP)
# solve_SCP!(TOSgusto, TOPgusto, solve_gusto_cvx!, init_traj_straightline, "Gurobi", OutputFlag=0) # Not working anymore

solve_SCP!(TOS_SCP, TOP, solve_gusto_jump!, init_traj_straightline, "Ipopt", print_level=0)

# solve_SCP!(TOSgusto, TOPgusto, solve_gusto_jump!, init_traj_nothing, "Ipopt", print_level=0)

In [ ]:
@show TOS_SCP.SCPS.converged
@show TOS_SCP.SCPS.iterations
@show TOS_SCP.SCPS.total_time
@show TOS_SCP.SCPS.accept_solution
@show TOS_SCP.SCPS.solver_status
@show TOS_SCP.SCPS.scp_status
@show TOS_SCP.SCPS.convergence_measure
@show TOS_SCP.SCPS.param.alg.ω_vec
@show TOS_SCP.SCPS.param.alg.Δ_vec
@show TOS_SCP.SCPS.J_true
@show TOS_SCP.SCPS.dual
;

In [ ]:
using Plots
gr(fmt=:png)
plot()
for i = 7:10
    plot!([collect(1:N)],[TOS_SCP.SCPS.traj.X[i,:]],
        xlabel = "q",
        ylabel = "y",
        legend = :none)
end
plot!()

In [ ]:
# for i in 1:29
#     @show TOS_SCP.SCPS.SCPC.nonconvex_state_convexified_ineq[:ncsi_obstacle_avoidance_signed_distance_convexified].con_reference[:,(27,)][i]
# end

In [ ]:
# for i in 1:29
#     @show TOS_SCP.SCPS.SCPC.nonconvex_state_convexified_ineq[:ncsi_obstacle_avoidance_potential].con_reference[:,(1,)][i]
# end

In [ ]:
include("../src/GuSTO.jl")

In [ ]:
@show TOS_SCP.SCPS.dual;

In [ ]:
dt = tf_guess/(N-1)
tspan = (0., tf_guess)
SP = ShootingProblem(TOP, TOS_SCP.SCPS)
x0 = [TOP.PD.x_init; TOS_SCP.SCPS.dual]
prob = ODEProblem(shooting_ode!, x0, tspan, SP)
sol = DifferentialEquations.solve(prob, dtmin=dt, force_dtmin=true, saveat=dt);

In [ ]:
X_ODE = zeros(13,N)
for i = 1:N
    X_ODE[:,i] = sol.u[i][1:13]
end
X_ODE;

In [ ]:
plot()
gr(fmt=:png)
for i = 7:10
    plot!([collect(1:N)],[X_ODE[i,:]],
        xlabel = "q",
        ylabel = "y",
        legend = :none)
end
plot!()

In [ ]:
plot()
gr(fmt=:png)
plot!([collect(1:N)],[X_ODE[1,:]],
    xlabel = "q",
    ylabel = "y",
    legend = :none)

In [ ]:
TOS_SCPS  = TrajectoryOptimizationSolution(TOP)
solve_SCPshooting!(TOS_SCPS, TOP, solve_gusto_jump!, init_traj_straightline, "Ipopt", print_level=0)
# solve_SCPshooting!(TOS_SCPS, TOP, solve_gusto_jump!, init_traj_straightline, "Gurobi")

In [ ]:
@show TOS_SCPS.SCPS.converged
@show TOS_SCPS.SCPS.iterations
@show TOS_SCPS.SCPS.total_time
@show TOS_SCPS.SCPS.accept_solution
@show TOS_SCPS.SCPS.solver_status
@show TOS_SCPS.SCPS.scp_status
@show TOS_SCPS.SCPS.convergence_measure
@show TOS_SCPS.SCPS.param.alg.ω_vec
@show TOS_SCPS.SCPS.param.alg.Δ_vec
@show TOS_SCPS.SCPS.J_true
;

In [ ]:
# SCP
@show TOS_SCPS.SCPS.convergence_measure
# Shooting
@show TOS_SCPS.SS.converged
@show TOS_SCPS.SS.prob_status
@show TOS_SCPS.SCPS.iterations
@show TOS_SCPS.total_time
@show TOS_SCPS.SS.iter_elapsed_times
@show TOS_SCPS.SS.J_true[end]

In [ ]:
plot()
gr(fmt=:png)
for i = 7:10
    plot!([collect(1:N)],[TOS_SCPS.SS.traj.X[i,:]],
        xlabel = "q",
        ylabel = "y",
        legend = :none)
end
plot!()

In [ ]:
plot()
gr(fmt=:png)
plot!([collect(1:N)],[TOS_SCPS.SS.traj.X[1,:]],
    xlabel = "q",
    ylabel = "y",
    legend = :none)

In [ ]:
using AstrobeeRobot

# Animate Astrobee trajectory
vis = Visualizer()
delete!(vis)

vis[:goal]
for (idx,obs) in enumerate(env.keepout_zones)
    setobject!(vis[:goal][:goal], 
        Object(HyperSphere(Point3f0(x_goal[1:3]), 0.1f0),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))
end

vis[:workspace]
for (idx,ws) in enumerate(env.keepin_zones)
    if idx in (5,8)
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3), depthWrite=false)))
    else
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3))))
    end
end

vis[:obstacle]
for (idx,ws) in enumerate(env.obstacle_set)
    setobject!(vis[:workspace][Symbol(string("ws",idx+length(env.keepin_zones)))],
        Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.26,0.26,0.3))))
end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);

speed_factor = 3

Qs = Vector{Vector{Float64}}(0)
for k in 1:speed_factor:N
#     q = [quat_inv(TOS_SCP.SCPS.traj.X[7:10,k]); TOS_SCP.SCPS.traj.X[1:3,k]]
    q = [quat_inv(TOS_SCPS.SS.traj.X[7:10,k]); TOS_SCPS.SS.traj.X[1:3,k]]
    push!(Qs,q)
end

trans = Translation(14., -1., 7.)
rot = LinearMap(RotZ(-0.6)) ∘ LinearMap(RotY(-0.2))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.9)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

sleep(3)
setanimation!(mvis,1:length(Qs),Qs)

plot_in_cell = false
plot_in_cell ? IJuliaCell(vis) : open(vis)

In [ ]:
x_sol = JuMP.value.(x[1,:])
# y_sol = JuMP.value.(x[2,:])


In [ ]:
p0 = -JuMP.dual.(con_init)

In [ ]:
# @show TOS_SCPS.SCPS.dual
dt = tf_guess/(N-1)
tspan = (0., tf_guess)
SP = ShootingProblem(TOP, TOS_SCPS.SCPS)
x0 = [x_init; p0]
prob = ODEProblem(shooting_ode!, x0, tspan, SP)
sol = DifferentialEquations.solve(prob, dtmin=dt, force_dtmin=true, saveat=dt);

In [ ]:
X_ODE = zeros(13,N)
for i = 1:N
    X_ODE[:,i] = sol.u[i][1:13]
end
X_ODE
# size(sol.u[end])

In [ ]:
x[:,1]

In [ ]:
function astrobee_dynamics(x, u, model, robot, j)
    x_dim, u_dim = model.x_dim, model.u_dim
    mass = robot.mass
    F, M = u[1:3], u[4:6]
    qx, qy, qz, qw = x[7:10]
    ωx, ωy, ωz = x[11:13]
    con_vec = Vector(x_dim)
    
    con_vec[1:3] = x[4:6]
    con_vec[4:6] = F/mass
    con_vec[7] = 1/2*( ωz*qy - ωy*qz + ωx*qw)
    con_vec[8] = 1/2*(-ωz*qx + ωx*qz + ωy*qw)
    con_vec[9] = 1/2*( ωy*qx - ωx*qy + ωz*qw)
    con_vec[10] = 1/2*(-ωx*qx - ωy*qy - ωz*qz)
    con_vec[11:13] = M
    return con_vec[j]
end
    
    

In [ ]:
Vector(4)